In [10]:
from __future__ import print_function
import os
import sys

In [11]:
#sc = SparkContext()
#sqlContext = SQLContext(sc)
sc

In [12]:
df = sqlContext.read.load("/tmp/idigbio-media-scrap-20170611T144515.parquet")

In [13]:
print(df.count())
df.head(2)

55


[Row(accessuri='http://osuc.osu.edu/spm_images/OSAL 0103928/73750_compress.jpg', datemodified=datetime.datetime(2015, 5, 6, 0, 42, 33, 923000), dqs=0.6363636255264282, etag='5140c42d67543dd47e16de943ce07b8a53bf1dc2', flags=None, format='image/jpeg', hasSpecimen=True, licenselogourl='https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png', mediatype='images', modified=datetime.datetime(2015, 2, 9, 10, 47, 33), recordset='95773ebb-2f5f-43f0-a652-bfd8d5f4707a', rights='BY-NC-SA', tag=None, type=None, uuid='17e3d421-3122-4839-86fd-4748edb003b2', version=None, webstatement='http://creativecommons.org/licenses/by-nc-sa/4.0/', xpixels=None, ypixels=None),
 Row(accessuri='http://osuc.osu.edu/spm_images/OSAL 0104620/73739_compress.jpg', datemodified=datetime.datetime(2015, 5, 6, 0, 42, 33, 957000), dqs=0.6363636255264282, etag='9510295daa95c8779bbd0665b62a972e64d6f706', flags=None, format='image/jpeg', hasSpecimen=True, licenselogourl='https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png', me

In [14]:
df.printSchema()

root
 |-- accessuri: string (nullable = true)
 |-- datemodified: timestamp (nullable = true)
 |-- dqs: float (nullable = true)
 |-- etag: string (nullable = true)
 |-- flags: string (nullable = true)
 |-- format: string (nullable = true)
 |-- hasSpecimen: boolean (nullable = true)
 |-- licenselogourl: string (nullable = true)
 |-- mediatype: string (nullable = true)
 |-- modified: timestamp (nullable = true)
 |-- recordset: string (nullable = true)
 |-- rights: string (nullable = true)
 |-- tag: string (nullable = true)
 |-- type: string (nullable = true)
 |-- uuid: string (nullable = true)
 |-- version: integer (nullable = true)
 |-- webstatement: string (nullable = true)
 |-- xpixels: integer (nullable = true)
 |-- ypixels: integer (nullable = true)



In [15]:
from pyspark.sql.types import *
def mk_field_set_from_df(df_schema):
    """Create a set of fieldname::type strings from a df schema"""
    field_set = set()
    prefix = ""
    for i in df_schema:
        #print(i)
        t = type(i.dataType)
        if t is not StringType and \
           t is not FloatType and \
           t is not IntegerType and \
           t is not TimestampType and \
           t is not BooleanType:
            #print(i.dataType)
            prefix = i.name
            
            # This only accommodates one level of nesting which is
            # consistant with our parquet-building technique. The
            # string manipulation on type makes it consistant with
            # the JSON returned by the API.
            for j in i.dataType:
                field_set.add(("{0}.{1}::{2}".format(prefix, j.name, 
                                            str(j.dataType)[:-4].lower())))

        else:
            field_set.add(("{0}::{1}".format(i.name, 
                                            str(i.dataType)[:-4].lower())))
    return field_set
df_field_set = mk_field_set_from_df(df.schema)
#print(df_field_set)

In [16]:
import requests

def mk_field_set_from_api():
    """Create a set of fieldname::type strings from the meta API endpoint"""
    meta_fields_records = (requests                                                                                                 
        .get("http://search.idigbio.org/v2/meta/fields/mediarecords")                                                 
        .json()                                                                                                  
    ) 
    field_set = set()                                                                                                              
    for k,v in meta_fields_records.items():
        if k == "data":                                                                                                             
            for kd,vd in v.items():                                                                                                 
                if vd.get("fieldName", False):                                                                                      
                    field_set.add("{0}::{1}".format(vd["fieldName"],
                                                        vd["type"]))
        elif v.get("fieldName", False):                                                                                               
            field_set.add("{0}::{1}".format(v["fieldName"], v["type"]))
        else:
            # non-data nested structures, only add top level key
            field_set.add("{0}::COMPLEX".format(k))
        
    return field_set
meta_field_set = mk_field_set_from_api()
#print(meta_field_set)

In [17]:
# The loaded df is a working data frame of course, what fields are in the
# meta endpoint that the df does not have? Are those fields currently 
# excluded in the parquet generation code or has Alex snuck in new
# structures lately?
#
# Note that ES calls things strings if they're lists of strings so even if
# something doesn't show up here, watch for scaler -> vector changes in the
# way data is represented in a field.
print("In meta endpoint, not df")
print("\n".join(sorted(meta_field_set - df_field_set)))
print("\nIn df, not meta endpoint")
print("\n".join(sorted(df_field_set - meta_field_set)))

In meta endpoint, not df
data.Iptc4xmpExt:CountryCode::string
data.Iptc4xmpExt:CountryName::string
data.Iptc4xmpExt:ProvinceState::string
data.Iptc4xmpExt:WorldRegion::string
data.ac:accessURI::string
data.ac:associatedSpecimenReference::string
data.ac:attributionLogoURL::string
data.ac:bestQualityAccessURI::string
data.ac:bestQualityFormat::string
data.ac:caption::string
data.ac:captureDevice::string
data.ac:comments::string
data.ac:digitizationDate::string
data.ac:furtherInformationURL::string
data.ac:goodQualityAccessURI::string
data.ac:hashFunction::string
data.ac:hashValue::string
data.ac:licenseLogoURL::string
data.ac:metadataCreator::string
data.ac:metadataLanguage::string
data.ac:metadataLanguageLiteral::string
data.ac:metadataProvider::string
data.ac:metadataProviderLiteral::string
data.ac:provider::string
data.ac:providerID::string
data.ac:providerLiteral::string
data.ac:providerManagedID::string
data.ac:resourceCreationTechnique::string
data.ac:serviceExpectation::string
dat